In [ ]:
!git clone https://github.com/Programacao-Paralela-e-Distribuida/MPI

Cloning into 'MPI'...
remote: Enumerating objects: 146, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 146 (delta 10), reused 5 (delta 5), pack-reused 120 (from 1)
Receiving objects: 100% (146/146), 3.94 MiB | 19.60 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [ ]:
!mv MPI/* .
!rm -rf MPI
!ls

mv: cannot move 'MPI/bin' to './bin': Directory not empty
mv: cannot move 'MPI/docs' to './docs': Directory not empty
mv: cannot move 'MPI/src' to './src': Directory not empty
bin  docs  Makefile  README.md	sample_data  src


#Broadcast

Um padrão de comunicação que envolva todos os processos em um comunicador é chamada de comunicação coletiva. Uma difusão (broadcast) é uma comunicação coletiva na qual um único processo envia os mesmos dados para cada processo.


**int MPI_Bcast (void* mensagem, int cont, MPI_Datatype tipo_mpi, int raiz, MPI_Comm com)**


A função **MPI_Bcast()** envia uma cópia dos dados de mensagem  no processo raiz para cada processo no mesmo comunicador. Para que a operação funcione corretamente, ela deve ser chamado por todos os processos no comunicador com os mesmos argumentos para *raiz* e *comunicador*.

Uma mensagem de difusão não pode ser recebida com **MPI_Recv()**, pois a operação é exclusivamente coletiva. Os parâmetros cont e tipo_mpi  têm a mesma função que nas funções **MPI_Send()** e **MPI_Recv()**, definindo o tamanho e o tipo da mensagem.

Contudo, ao contrário das funções ponto-a-ponto, o padrão MPI exige que os valores dps parâmetros do total e tipo dos elementos enviados sejam  iguais  para todos os processos dentro do mesmo comunicador em uma comunicação coletiva.

No exemplo a seguir, o valor recebido pelo processo raiz é enviado para todos os processos no comunicador.



In [ ]:
!cat src/mpi_bcast.c

#include <stdio.h>
#include "mpi.h"

int main(int argc, char *argv[]) { /* mpi_bcast.c  */
int valor, meu_ranque, raiz = 0;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
    /* Cada processo tem um valor inicial diferente */
    valor = meu_ranque;
    /* O valor a ser enviado é lido pelo processo raiz */
    if (meu_ranque == raiz) {
        printf("Entre um valor: \n");
        scanf("%d", &valor);    
    }
    /* A rotina de difusão é chamada por todos processos, apenas o processo raiz envia, os demais recebem */
    MPI_Bcast(&valor, 1, MPI_INT, raiz, MPI_COMM_WORLD);
    /* O valor agora é o mesmo em todos os processos */
    printf("O processo com ranque %d recebeu o valor: %d\n",meu_ranque, valor);
    /* Termina a execução */
    MPI_Finalize();
    return 0;
}


In [ ]:
!mpicc -o bin/mpi_bcast src/mpi_bcast.c
!echo "33" | mpirun --allow-run-as-root -np 4 bin/mpi_bcast

O processo com ranque 2 recebeu o valor: 33
O processo com ranque 3 recebeu o valor: 33
Entre um valor: 
O processo com ranque 0 recebeu o valor: 33
O processo com ranque 1 recebeu o valor: 33


#Coleta
A operação de coleta é realizada pela função **MPI_Gather()**.

**int MPI_Gather(void* vet_envia, int cont_envia, MPI_Datatype tipo_envia, void* vet_recebe, int cont_recebe, MPI_Datatype tipo_recebe, int raiz, MPI_comm com)**

Cada processo no comunicador *com* envia o conteúdo de *vet_envi*a para o processo com ranque igual a *raiz*.  O processo raiz concatena os dados que são recebidos em *vet_recebe*  em uma ordem que é definida pelo ranque de cada processo.

Os argumentos que terminam com *recebe* são significativos apenas no processo com ranque igual a raiz. O argumento *cont_recebe* indica o número de itens enviados por cada processo, não número total de itens recebidos pelo processo raiz e, normalmente, é igual ao argumento *cont_envia*.

No exemplo a seguir, o processo raiz recebe um vetor com 10 elementos de cada processo (dele mesmo inclusive) e concatena em um vetor de *num_procs* $\times$10  elementos e imprime o resultado.

In [ ]:
!mpicc -o bin/mpi_gather src/mpi_gather.c
!mpirun --allow-run-as-root -np 4 bin/mpi_gather

Processo =  0, recebeu 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3


In [ ]:
!cat src/mpi_gather.c

#include <stdio.h>
#include <stdlib.h>
#include "mpi.h"
#define TAM_VET 10

int main(int argc, char *argv[]) { /* mpi_gather.c  */
int meu_ranque, num_procs, i, raiz = 0;
int *vet_recebe,  vet_envia[TAM_VET];
	
	MPI_Init(&argc, &argv);
	MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
	MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
	/* O processo raiz aloca o espaço de memória para o vetor de recepção */
	if (meu_ranque == raiz)
        vet_recebe = (int *) malloc(num_procs*TAM_VET*sizeof(int)); 
    /* Cada processo atribui valor inicial ao vetor de envio */
	for (i = 0; i < TAM_VET; i++) 
         vet_envia[i] = meu_ranque;
    /* O vetor é recebido pelo processo raiz com as partes recebidas de todos processos, incluindo o processo raiz */
	MPI_Gather (vet_envia, TAM_VET, MPI_INT, vet_recebe, TAM_VET, MPI_INT, raiz, MPI_COMM_WORLD);
	/* O processo raiz imprime o vetor recebido */ 
	if (meu_ranque == raiz) {
	    printf("Processo =  %d, recebeu", meu_ranque);
	    for (i = 0; i < (TAM_VET*num_


#Distribuição

A operação de distribuição (scatter) é realizada pela função MPI_Scatter() do MPI.

**int MPI_Scatter(void* vet_envia, int cont_envia, MPI_Datatype tipo_envia, void* vet_recebe, int cont_recebe, MPI_Datatype tipo_recebe, int raiz, MPI_Comm com)**

O processo com o ranque igual a raiz distribui o conteúdo de *vet_envia* entre os *p* processos.  O conteúdo de *vet_envia* é dividido em *p* segmentos, cada um deles consistindo de *cont_envia* itens.
O primeiro segmento vai para o processo com ranque 0, o segundo para  o processo com ranque  1, e assim por diante. Os argumentos que terminam com *envia* são significativos apenas no processo raiz, pois ele é o único que possui o conteúdo completo a ser distribuído.

O exemplo a seguir o processo raiz monta um vetor com *num_procs* $\times$10  elementos e envia um vetor de  10 elementos para cada processo, ele mesmo incluído.


In [ ]:
!cat src/mpi_scatter.c

#include <stdio.h>
#include <stdlib.h>
#include "mpi.h"
#define TAM_VET 10

int main(int argc, char *argv[]) { /* mpi_scatter.c  */
int i, meu_ranque, num_procs, raiz = 0;
int *vet_envia, vet_recebe[TAM_VET];
	
	MPI_Init(&argc, &argv);
    	MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
    	MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
    	/* O processo raiz aloca o espaço de memória e inicia o vetor */
    	if (meu_ranque == raiz) {
        	vet_envia = (int*) malloc (num_procs*TAM_VET*sizeof(int));
        	for (i = 0; i < (TAM_VET*num_procs); i++) 
             		vet_envia[i] = i;
    	}
    	/* O vetor é distribuído em partes iguais entre os processos, incluindo o processo raiz */
    	MPI_Scatter(vet_envia, TAM_VET, MPI_INT, vet_recebe, TAM_VET, MPI_INT, raiz, MPI_COMM_WORLD);
	/* Cada processo imprime a parte que recebeu */ 
	printf("Processo =  %d, recebeu", meu_ranque);
	for (i = 0; i < TAM_VET; i ++) 
		printf(" %d", vet_recebe[i]);
	printf("\n");
	/* Termina a execução */
	MPI_Fi

In [ ]:
!mpicc -o bin/mpi_scatter src/mpi_scatter.c
!mpirun --allow-run-as-root -np 4 bin/mpi_scatter

Processo =  0, recebeu 0 1 2 3 4 5 6 7 8 9
Processo =  1, recebeu 10 11 12 13 14 15 16 17 18 19
Processo =  2, recebeu 20 21 22 23 24 25 26 27 28 29
Processo =  3, recebeu 30 31 32 33 34 35 36 37 38 39


#Redução

Em uma operação global de redução, todos os processos em um comunicador contribuem com dados que são combinados em operações binárias.
Operações binárias típicas são a adição, máximo, mínimo, e lógico, etc.
É possível definir operações adicionais além das mostradas para a função **MPI_Reduce()**.

**int MPI_Reduce(void* operando, void* resultado, int cont, MPI_Datatype tipo_mpi, MPI_Op oper, int raiz, MPI_Comm com)**

A operação MPI_Reduce() combina os operandos armazenados em *operando* usando a operação *oper* e armazena o resultado em *resultado*  no processo raiz. Tanto *operando* como *resultado* referem-se a *cont* posições de memória com o tipo *tipo_mpi*.

**MPI_Reduce()** deve ser chamada por todos os processos no comunicador *com*, os valores de *cont*, *tipo_mpi* e *oper* devem ser os mesmos em  todos processos.

O exemplo a seguir calcula o valor máximo para cada uma das 5 posições de um vetor.

In [ ]:
!cat src/mpi_reduce.c

#include <stdio.h> 
#include <math.h>
#include "mpi.h" 
#define TAM 5

int main(int argc, char *argv[]) { /* mpi_reduce.c  */
int meu_ranque, num_procs, i, raiz = 0;      
float vet_envia [TAM] ; /* Vetor a ser enviado  */ 
float vet_recebe [TAM]; /* Vetor a ser recebido */  

    MPI_Init(&argc, &argv); 
    MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque); 
    MPI_Comm_size(MPI_COMM_WORLD, &num_procs); 
    /* Preenche o vetor com valores que dependem do ranque */
    for (i = 0; i < TAM; i++)  
        vet_envia[i] = (float) (meu_ranque*TAM+i);
    /* Faz a redução, encontrando o valor máximo do vetor */
    MPI_Reduce(vet_envia, vet_recebe, TAM, MPI_FLOAT, MPI_MAX, raiz, MPI_COMM_WORLD); 
    /* O processo raiz imprime o resultado da operação de redução */
    if (meu_ranque == raiz) { 
        for (i = 0; i < TAM; i++) 
            printf("vet_recebe[%d] = %3.1f ", i,vet_recebe[i]); 
        printf("\n\n");     } 
    MPI_Finalize();
    return(0);
}


In [ ]:
!mpicc -o bin/mpi_reduce src/mpi_reduce.c
!mpirun --allow-run-as-root -np 4 bin/mpi_reduce

vet_recebe[0] = 15.0 vet_recebe[1] = 16.0 vet_recebe[2] = 17.0 vet_recebe[3] = 18.0 vet_recebe[4] = 19.0 



#Redução Global

**int MPI_Allreduce (void* vet_envia, void* vet_recebe, int cont, MPI_Datatype tipo_mpi, MPI_Op oper, MPI_Comm com)**
<br>
<br>
**MPI_Allreduce()** tem um resultado similar à operação **MPI_Reduce()**, com a diferença que o resultado da operação de redução *oper* é armazenado no vetor *vet_recebe* de cada processo envolvido na chamada e não apenas no processo raiz. Aliás, como todos recebem o resultado da redução, não há necessidade do parâmetro raiz no protótipo da função.
<br>
<br>
Veja o exemplo a seguir, que calcula o valor aproximando de Pi, usando a aproximação pela fórmula:
<br>
<br>
$\pi = 4 \int_0^1 \frac{1}{1+x^2} \, dx.$

In [ ]:
!cat src/mpi_calcpi.c


#include<stdio.h>
#include <math.h> 
#include "mpi.h"
int main (int argc, char *argv[]){	
int meu_ranque, num_procs, n=10000000; 
double mypi, pi, h, x, sum = 0.0;
	MPI_Init(&argc, &argv);
	MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
	MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
	h=1.0/(double) n;
	for (int i = meu_ranque +1; i <= n; i += num_procs){	
		x = h * ((double) i - 0.5);
		sum  += (4.0/(1.0 + x*x));
	}
	mypi = h* sum;
	MPI_Allreduce(&mypi, &pi, 1, MPI_DOUBLE, MPI_SUM, MPI_COMM_WORLD);
	printf ("valor aproximado de pi: %.16f \n", pi);
	MPI_Finalize( );
}
